Eight Puzzle problem.  For example:

1 4 5                 1 2 3
2 3 7           =>    4 5 6
  6 7                 7 8   
  
Model board as [[1, 4, 5], [2,3,7], [0, 6, 7]]
Four possible moves are UP, DOWN, LEFT, RIGHT, referring to the blank space (0 element)


**TODO:  implement second heuristic, sum of manhattan distances to target**

In [34]:
from searchFramework import WorldState
import copy

class EightPuzzleWorldState(WorldState):

    def __init__(self, board):
        self._board = board
        
    def __str__(self):
        return "{" + str(self._board) + "}"
    
    def __eq__(self, other):
        if isinstance(other, EightPuzzleWorldState):
            return self._board == other._board
        else:
            return False

    def __hash__(self):
        return hash(str(self._board))
    
    # NB: every successor state must deep copy the old state!
    
    def successors(self):
        candidates = (self.up(), self.down(), self.left(), self.right())
        return [c for c in candidates if c] 
    
    def up(self):
        bp = self.blankPosition()
        if (bp[0] == 0):
            return None
        else:
            s = copy.deepcopy(self)
            s.swap(bp, (bp[0] -1, bp[1]))
            return((s, "up"))
                   
    def down(self):
        bp = self.blankPosition()
        if (bp[0] == self.boardSize() - 1):
            return None
        else:
            s = copy.deepcopy(self)
            s.swap(bp, (bp[0] + 1, bp[1]))
            return ((s, "down"))   
    def left(self):
        bp = self.blankPosition()
        if (bp[1] == 0):
            return None
        else:
            s = copy.deepcopy(self)
            s.swap(bp, (bp[0], bp[1] - 1))
            return ((s, "left"))
    def right(self):
        bp = self.blankPosition()
        if (bp[1] == self.boardSize() - 1):
            return None
        else:
            s = copy.deepcopy(self)
            s.swap(bp, (bp[0], bp[1] + 1))
            return ((s, "right"))

    def boardSize(self):
        return len(self._board[0])
    
    def swap(self, p1, p2):
        tmp = self._board[p1[0]][p1[1]]
        self._board[p1[0]][p1[1]] = self._board[p2[0]][p2[1]]
        self._board[p2[0]][p2[1]] = tmp
    
    def blankPosition(self):
        for i in range(self.boardSize()):
            for j in range(self.boardSize()):
                   if self._board[i][j] == 0:
                       return (i,j)
        return None

In [35]:
w = EightPuzzleWorldState([[1, 4, 5], [2,0,7], [3, 6, 7]])

In [3]:
ww = w.down()
str(ww[0])

'{[[1, 4, 5], [2, 6, 7], [3, 0, 7]]}'

In [41]:
from searchFramework import Problem

class EightPuzzleProblem(Problem):
    def __init__(self, initial_board, goal_board = [[1,2,3], [4,5,6], [7,8,0]]):
        self._state = EightPuzzleWorldState(initial_board)
        self._goal_board = goal_board
        
    def initial(self):
        return self._state
    
    def isGoal(self, state):
        return state._board == self._goal_board
        

In [42]:
p = EightPuzzleProblem([[1, 4, 5], [2,0,7], [3, 6, 7]])

In [43]:
p.initial()

In [44]:
p.isGoal(EightPuzzleWorldState([[1,2,3], [4,5,6], [7,8,0]]))

True

In [8]:
from searchFramework import Evaluator

In [9]:
## Breadth-first search -- make short paths look cheapest

def bfsCoster(actions):
    return len(actions)

## Cost to goal is number of tiles that are not in the right position

def bfsEstimator(state):
    return 0

bfsEvaluator = Evaluator(bfsCoster, bfsEstimator)

In [10]:
## Depth-first search -- make long paths look cheapest

def dfsCoster(actions):
    return -len(actions)

## Cost to goal is number of tiles that are not in the right position

def dfsEstimator(state):
    return 0

dfsEvaluator = Evaluator(dfsCoster, dfsEstimator)

In [23]:
# Num tiles heuristic

def numTilesCoster(actions):
    return len(actions)

## Cost to goal is number of tiles that are not in the right position

def numTilesEstimator(state):
    #gb = [[1,2,3], [4,5,6], [7,8,0]]
    gb = [[1,2,3], [8,0,4], [7,6,5]]
    est = 0
    for i in range(state.boardSize()):
        for j in range(state.boardSize()):
            if state._board[i][j] != gb[i][j]:
                est += 1
    return est

numTilesEvaluator = Evaluator(numTilesCoster, numTilesEstimator)

In [24]:
# Num tiles heuristic

def numTilesCoster(actions):
    return len(actions)

## Cost to goal is number of tiles that are not in the right position

def tileDistanceEstimator(state):
    #gb = [[1,2,3], [4,5,6], [7,8,0]]
    gb = [[1,2,3], [8,0,4], [7,6,5]]
    est = 0
    for i in range(state.boardSize()):
        for j in range(state.boardSize()):
            est += distanceFrom(state._board, i,j, gb)
    return est

def distanceFrom(board, i, j, goalboard):
    tileValue = board[i][j]
    goalPos = positionOf(tileValue, goalboard)
    return abs(i - goalPos[0]) + abs(j - goalPos[1])

def positionOf(value, board):
    for i in range(len(board)):
        for j in range(len(board)):
            if board[i][j] == value:
                return (i,j)
            
tileDistanceEvaluator = Evaluator(numTilesCoster, tileDistanceEstimator)

In [13]:
easyBoard = [[1,2,3], [4,5,6], [7,0,8]]
easyProblem = EightPuzzleProblem(easyBoard)

In [14]:
from searchFramework import aStarSearch

In [15]:
print(aStarSearch(easyProblem, bfsEvaluator))

(['right'], (0.0, 4, 0, 7))


In [16]:
# LOOPS
print(aStarSearch(easyProblem, dfsEvaluator, 1, 100))

Visited 1 world is {[[1, 2, 3], [4, 5, 6], [7, 0, 8]]}
Skipped 0 Fringe is size 0
Evaluation is 0 with actions 0
Visited 2 world is {[[1, 2, 3], [4, 0, 6], [7, 5, 8]]}
Skipped 0 Fringe is size 2
Evaluation is -1 with actions 1
Visited 3 world is {[[1, 0, 3], [4, 2, 6], [7, 5, 8]]}
Skipped 0 Fringe is size 5
Evaluation is -2 with actions 2
Visited 4 world is {[[1, 2, 3], [4, 0, 6], [7, 5, 8]]}
Skipped 0 Fringe is size 7
Evaluation is -3 with actions 3
Visited 5 world is {[[0, 1, 3], [4, 2, 6], [7, 5, 8]]}
Skipped 1 Fringe is size 6
Evaluation is -3 with actions 3
Visited 6 world is {[[4, 1, 3], [0, 2, 6], [7, 5, 8]]}
Skipped 1 Fringe is size 7
Evaluation is -4 with actions 4
Visited 7 world is {[[0, 1, 3], [4, 2, 6], [7, 5, 8]]}
Skipped 1 Fringe is size 9
Evaluation is -5 with actions 5
Visited 8 world is {[[4, 1, 3], [7, 2, 6], [0, 5, 8]]}
Skipped 2 Fringe is size 8
Evaluation is -5 with actions 5
Visited 9 world is {[[4, 1, 3], [0, 2, 6], [7, 5, 8]]}
Skipped 2 Fringe is size 9
Evaluat

In [17]:
print(aStarSearch(easyProblem, numTilesEvaluator))

(['right'], (0.0, 2, 0, 3))


In [18]:
print(aStarSearch(easyProblem, tileDistanceEvaluator))

(['right'], (0.0, 2, 0, 3))


In [19]:
b = [[1, 5,2], [7,4,3], [0, 8, 6]]
harderProblem = EightPuzzleProblem(b)

In [20]:
print(aStarSearch(harderProblem, bfsEvaluator))

(['up', 'right', 'up', 'right', 'down', 'down'], (0.0, 98, 36, 69))


In [ ]:
# Loops -- also 100000 takes a really long time!  Reminder that our search
# is pretty poorly implemented!

print(aStarSearch(harderProblem, dfsEvaluator, None, 1000))

In [21]:
# Works much better than bfs!
print(aStarSearch(harderProblem, numTilesEvaluator))

(['up', 'right', 'up', 'right', 'down', 'down'], (0.0, 7, 0, 12))


In [22]:
print(aStarSearch(harderProblem, tileDistanceEvaluator))

(['up', 'right', 'up', 'right', 'down', 'down'], (0.0, 8, 0, 14))


These examples came from a web page: http://www.d.umn.edu/~jrichar4/8puz.html
but the examples have a different goal state


In [26]:
class DifferentEightPuzzleProblem(Problem):
    def __init__(self, board):
        self._state = EightPuzzleWorldState(board)
        
    def initial(self):
        return self._state
    
    def isGoal(self, state):
        return state._board == [[1,2,3], [8,0,4], [7,6,5]]
        

In [27]:
b1 = [[1,3,4], [8,6,2], [7,0,5]]
b2 = [[2,8,1], [0,4,3], [7,6,5]]
b3 = [[2,8,1], [4,6,3], [0,7,5]]
b4 = [[5,6,7], [4,0,8], [3,2,1]]

In [32]:
print(aStarSearch(DifferentEightPuzzleProblem(b3), numTilesEvaluator, None, 25000))

(['right', 'up', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'down'], (0.015625, 174, 70, 125))


In [33]:
print(aStarSearch(DifferentEightPuzzleProblem(b3), tileDistanceEvaluator, None, 25000))

(['right', 'up', 'left', 'up', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'down'], (0.0, 44, 16, 36))
